### Loading the dataset

In [62]:
!pip uninstall jedi
from datasets import load_dataset
from pathlib import Path
import os

In [4]:
# reading the jsonl dataset
data_dir = Path("../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl")

import pandas as pd    
jsonObj = pd.read_json(path_or_buf=data_dir, lines=True)



In [5]:
jsonObj.head()["answer"][2]

'reject'

In [6]:
jsonObj.columns

Index(['text', 'meta', '_input_hash', '_task_hash', 'label', 'score',
       'priority', 'spans', '_session_id', '_view_id', 'answer'],
      dtype='object')

### Huggingface dataset loading script
can be used later to publish the dataset on huggingface hub for better outreach

### Testing the dataset loading script

In [31]:
from datasets import load_dataset


dataset = load_dataset("my_dataset_loading_script", name="condemnation")

Reusing dataset condemnation_dataset (/home/geev/.cache/huggingface/datasets/condemnation_dataset/condemnation/1.1.0/a430c0ac9660e8ba6ce252e564d88528d156f575bd9dbfe6e30bd1fa1c359008)


In [32]:
train_dataset = dataset["train"]
train_dataset.shape

(1680, 2)

In [33]:
train_dataset[0]

{'text': "Colorado's House Speaker (D) Crisanta Duran removes Rep. -JOHN DOE- from a position following a report of sexual harrassment",
 'label': 0}

In [34]:
train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [35]:
train_dataset.dataset_size

683445

In [36]:
# train_dataset.set_format(type="torch", columns=["label"])
train_dataset[:10]

OrderedDict([('text',
              ["Colorado's House Speaker (D) Crisanta Duran removes Rep. -JOHN DOE- from a position following a report of sexual harrassment",
               '⚡️ “NPR chief -JOHN DOE- resigns in light of harassment allegations” URL',
               "-JOHN DOE- regrets being 'really hurtful to women' - CNET",
               "Here we go again. Yet another LIBERAL accused of sexual harassment. This time it's -JOHN DOE- from none other than ultra-liberal #NPR. The lack of moral compass and integrity will be undoing of what is left of Democrat Party. #MAGA #WednesdayWisdom #tcot #WorldVeganDay URL",
               'I remember when everyone was dickriding -JOHN DOE- as a "great writer" and "the only decent IGN employee". I never understood it and got lots of shit for saying he always came off as a weird pervert in videos. Fucking called it years ago. And his writing was pure garbage.',
               "I'm a Matt Damon fan, can't even believe he was friends w -JOHN DOE- 

In [59]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def encode(batch):
    return tokenizer(batch["text"], padding="longest", truncation=True, max_length=512, return_tensors="pt")
# train_dataset.set_format(type="torch", columns=["label"], output_all_columns=True)
train_dataset.set_transform(encode, columns=["text"], output_all_columns=True)


In [60]:
train_dataset[0]

{'input_ids': tensor([  101,  5169,  1005,  1055,  2160,  5882,  1006,  1040,  1007, 13675,
         29196,  2696, 22959, 20362, 16360,  1012,  1011,  2198, 18629,  1011,
          2013,  1037,  2597,  2206,  1037,  3189,  1997,  4424,  5292, 11335,
          4757,  3672,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'label': 0}

In [43]:
train_dataset.reset_format()
train_dataset[0]

{'text': "Colorado's House Speaker (D) Crisanta Duran removes Rep. -JOHN DOE- from a position following a report of sexual harrassment",
 'label': 0}

### Selecting the training set

In [82]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
data_dir = "my_dataset_loading_script"
file_name = "nour_jillian_condemnation_r2_to_r7_maj_vote.jsonl"
def create_train_test(data_dir, file_name):
    file_path = os.path.join(data_dir, file_name )
    ds_df = pd.read_json(file_path, lines=True)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    sss.get_n_splits(ds_df, ds_df["label"])
    print(sss)
#     print(ds_df)
    for train_index, test_index in sss.split(ds_df, ds_df["label"]):
        print(train_index)
        train_df= ds_df.iloc[train_index,:]
        test_df= ds_df.iloc[test_index,:]

#         test_df = ds_df[test_index]
    return train_df, test_df
train_df, test_df = create_train_test(data_dir, file_name)
        

StratifiedShuffleSplit(n_splits=1, random_state=0, test_size=0.2,
            train_size=None)
[1509  415  988 ...  864 1143   91]


In [85]:
sum(train_df.label)/len(train_df.label)

0.6264880952380952

In [86]:
sum(test_df.label)/len(test_df.label)

0.6279761904761905